In [21]:
from pprint import pprint
from proxy import Resolve

r = Resolve('/Volumes/thom_7_for_Mac/tmp/素材')

# r.project.GetSetting()

# def project_color_setting():
#     r.project.SetSetting('colorScienceMode', 'davinciYRGBColorManagedv2')
#     r.project.SetSetting('colorSpaceTimeline', 'DaVinci WG/Intermediate')
#     r.project.SetSetting('colorSpaceOutput', 'Rec.709 Gamma 2.4')
#     r.project.SetSetting('timelineWorkingLuminanceMode', 'SDR 100')
#     r.project.SetSetting('timelineWorkingLuminanceMode', 'SDR 100')
#     r.project.SetSetting('inputDRT', 'DaVinci')
#     r.project.SetSetting('outputDRT', 'DaVinci')
#     r.project.SetSetting('useColorSpaceAwareGradingTools', '1')

# r.timeline.GetSetting()

for subfolder in r.root_folder.GetSubFolderList():
    for clip in subfolder.GetClipList():
        # clip.SetClipProperty('Input Color Space', 'DJI D-Gamut/D-Log')
        pprint(clip.GetClipProperty('File Path'))

'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/FX6#3/CARD#B/A008C0076_220620_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/FX6#3/CARD#A/A008C0081_220620_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#1/CARD#A/A008C0101_220620_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#1/CARD#A/A010C0037_220622_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#1/CARD#A/A010C0044_220622_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#2/CARD#C/220709_FX3_A_0083.MP4'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#2/CARD#A/A011C0010_220623_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#2/CARD#A/A011C0018_220623_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#2/CARD#A/A011C0022_220623_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ronin_4D#2/CARD#A/A011C0033_220623_005G.MOV'
'/Users/thom/Movies/Videos/Footages/悠长假期/素材/Day_01/Ron